In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error

# Merging CSV for Feature selection 

In [2]:
players_df = pd.read_csv('players.csv')
plays_df =pd.read_csv('plays.csv')
tackles_df = pd.read_csv('tackles.csv')
tracking_week_1_df = pd.read_csv('tracking_week_1.csv')

In [3]:
tackles_df = pd.merge(players_df,tackles_df,on=['nflId'], how='left')
print(tackles_df.shape)

(18309, 13)


In [4]:
final_df = pd.merge(tackles_df, tracking_week_1_df, on=['playId','gameId','nflId'], how='left')
print(final_df.shape)


(109259, 27)


In [5]:
final_df = pd.merge(plays_df,final_df,on=['playId','gameId'], how='left')
print(final_df.shape)

(108837, 60)


In [6]:
final_df.columns

Index(['gameId', 'playId', 'ballCarrierId', 'ballCarrierDisplayName',
       'playDescription', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'defensiveTeam', 'yardlineSide', 'yardlineNumber', 'gameClock',
       'preSnapHomeScore', 'preSnapVisitorScore', 'passResult', 'passLength',
       'penaltyYards', 'prePenaltyPlayResult', 'playResult',
       'playNullifiedByPenalty', 'absoluteYardlineNumber', 'offenseFormation',
       'defendersInTheBox', 'passProbability', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded',
       'visitorTeamWinProbilityAdded', 'expectedPoints', 'expectedPointsAdded',
       'foulName1', 'foulName2', 'foulNFLId1', 'foulNFLId2', 'nflId', 'height',
       'weight', 'birthDate', 'collegeName', 'position', 'displayName_x',
       'tackle', 'assist', 'forcedFumble', 'pff_missedTackle', 'displayName_y',
       'frameId', 'time', 'jerseyNumber', 'club', 'playDirection', 'x', 'y',
       's', 'a', 'di

# Feature Selection 

In [7]:
final_data = pd.DataFrame()
columns =['yardsToGo','yardlineNumber','passLength','absoluteYardlineNumber','position','playDirection','dis','o','dir','x','y']
final_data[columns] = final_df[columns]
print(final_data)

        yardsToGo  yardlineNumber  passLength  absoluteYardlineNumber  \
0              10              41         6.0                      69   
1              10              34         NaN                      76   
2              10              34         NaN                      76   
3              10              34         NaN                      76   
4              10              34         NaN                      76   
...           ...             ...         ...                     ...   
108832         10              41         NaN                      69   
108833         10              34         NaN                      44   
108834         10              34         NaN                      44   
108835          1              17         NaN                      27   
108836          1              17         NaN                      27   

       position playDirection   dis       o     dir      x      y  
0            FS           NaN   NaN     NaN     NaN    

# Label Encoding for categorical features 

In [8]:
label_encoder = LabelEncoder()
# Fit and transform the categorical column
final_data['position'] = label_encoder.fit_transform(final_data['position'])

In [9]:
final_data['playDirection'] = label_encoder.fit_transform(final_data['playDirection'])

In [10]:
final_data.rename(columns={'x': 'cord1', 'y': 'cord2'}, inplace=True)# Renaming X&Y and cord1 , cord2

In [11]:
final_data=final_data.dropna()   # droping Null values 
final_data = final_data.reset_index(drop=True)

In [12]:
final_data

,yardsToGo,yardlineNumber,passLength,absoluteYardlineNumber,position,playDirection,dis,o,dir,cord1,cord2
0,5,30,11.0,40,4,1,0.20,267.13,279.69,57.89,26.88
1,5,30,11.0,40,4,1,0.27,270.92,283.11,57.63,26.95
2,5,30,11.0,40,4,1,0.33,272.04,287.08,57.32,27.05
3,5,30,11.0,40,4,1,0.40,272.04,292.74,56.95,27.20
4,5,30,11.0,40,4,1,0.44,272.65,296.43,56.56,27.40
...,...,...,...,...,...,...,...,...,...,...,...
26832,10,30,0.0,40,4,1,0.34,0.30,358.37,41.94,18.59
26833,10,30,0.0,40,4,1,0.29,353.56,356.64,41.93,18.88
26834,10,30,0.0,40,4,1,0.24,2.14,357.57,41.92,19.12
26835,10,30,0.0,40,4,1,0.18,3.66,357.42,41.91,19.30


# Predicting CORD1(X)

In [13]:
X = final_data.iloc[:,0:-2].values  #defining X & Y 
y = final_data.iloc[:,-2].values

In [14]:
scaler=StandardScaler()   #Standard scaling of features & target variable 
X = scaler.fit_transform(X)    

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  #spliting X & Y 

In [16]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)    #model fitting / training 

RandomForestRegressor(random_state=42)

In [17]:
prediction_X = rf_regressor.predict(X_test) #here y_pred is X coordinate of location  

In [41]:
# checking error values
mae = mean_absolute_error(y_test, prediction_X)
print(f'Mean Absolute Error: {mae}')
mse = mean_squared_error(y_test, prediction_X)
print(f'Mean Square Error: {mse}')
rmse = np.sqrt(mse1)
print(f'Root Mean Square Error: {rmse}')
r2 = r2_score(y_test, prediction_X)
print(f'R2 score: {r2}')
medae = median_absolute_error(y_test, prediction_X)
print(f'Median Absolute Error: {medae}')

Mean Absolute Error: 0.6482049552906105
Mean Square Error: 1.5738107079955272
Root Mean Square Error: 0.000517532107256438
R2 score: 0.9969178589540111
Median Absolute Error: 0.29964999999999975


In [19]:
mse,rmse,mae,r2,medae

(1.5738107079955272,
 1.2545161250440455,
 0.6482049552906105,
 0.9969178589540111,
 0.29964999999999975)

# Predicting cord2(Y)

In [20]:
data=final_data.copy()  #creating copy of final_data as data 

In [21]:
X_pred = rf_regressor.predict(X)  

In [22]:
data['cord1_pred']=X_pred   #adding prediction X variable to dataframe 

In [23]:
data

,yardsToGo,yardlineNumber,passLength,absoluteYardlineNumber,position,playDirection,dis,o,dir,cord1,cord2,cord1_pred
0,5,30,11.0,40,4,1,0.20,267.13,279.69,57.89,26.88,57.3630
1,5,30,11.0,40,4,1,0.27,270.92,283.11,57.63,26.95,57.4642
2,5,30,11.0,40,4,1,0.33,272.04,287.08,57.32,27.05,57.2313
3,5,30,11.0,40,4,1,0.40,272.04,292.74,56.95,27.20,56.8971
4,5,30,11.0,40,4,1,0.44,272.65,296.43,56.56,27.40,56.4795
...,...,...,...,...,...,...,...,...,...,...,...,...
26832,10,30,0.0,40,4,1,0.34,0.30,358.37,41.94,18.59,45.2524
26833,10,30,0.0,40,4,1,0.29,353.56,356.64,41.93,18.88,42.0175
26834,10,30,0.0,40,4,1,0.24,2.14,357.57,41.92,19.12,43.5075
26835,10,30,0.0,40,4,1,0.18,3.66,357.42,41.91,19.30,43.2551


In [30]:
y1=data.iloc[:,-2].values    #fetching  Features and target variable 
X1=data.iloc[:,0:10].values
scaler=StandardScaler()
X1 = scaler.fit_transform(X1)  #Standard scaling of X & Y  
y1 = y.reshape(-1, 1)
y1= scaler.fit_transform(y1)

In [31]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)   #spiliting of X & Y 

In [32]:
rf_regressor1 = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor1.fit(X_train1, y_train1)    #fitting/training of model 

C:\Users\sharmashailza\AppData\Local\Temp\ipykernel_10352\448746688.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_regressor1.fit(X_train1, y_train1)


RandomForestRegressor(random_state=42)

In [33]:
prediction_Y = rf_regressor1.predict(X_test1)    #Here y_pred1 is Y coordinate of location 

In [39]:
# checking error values
mae1 = mean_absolute_error(y_test1, prediction_Y)
print(f'Mean Absolute Error: {mae1}')
mse1 = mean_squared_error(y_test1, prediction_Y)
print(f'Mean Square Error: {mse1}')
rmse1 = np.sqrt(mse1)
print(f'Root Mean Square Error: {rmse1}')
r22 = r2_score(y_test1, prediction_Y)
print(f'R2 score: {r22}')
medae1 = median_absolute_error(y_test1, prediction_Y)
print(f'Median Absolute Error: {medae1}')

Mean Absolute Error: 0.0001592267338749513
Mean Square Error: 2.678394820412893e-07
Root Mean Square Error: 0.000517532107256438
R2 score: 0.9999997400661597
Median Absolute Error: 7.187467201519304e-05


0.6482049552906105